# Preprocessing the Main Dataset
In the following file we pre-process the Amazon Movie Reviews Dataset, filtering out all bad reviews, the reviewers with only one review. Furthermore, we merge this dataset with the categories the movies were found in (additional dataset retrieved from GitHub). Finally, we reduce the amount of data to about 10% of the total data for computational reasons, by selecting five distinct genres.

In [70]:
import re
import gzip
import shutil
import random
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

from time import sleep
from tqdm import tqdm

In [ ]:
# Writing the text file from the .gz format of the dataset
with gzip.open('movies.txt.gz', 'rb') as f_in:
    with open('movies.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [2]:
# Opening the text file
with open('movies.txt', errors='ignore') as f:
    lines = f.readlines()

In [4]:
# Quick view into the file
print(len(lines))
lines[:9]

71205231


['product/productId: B003AI2VGA\n',
 'review/userId: A141HP4LYPWMSR\n',
 'review/profileName: Brian E. Erland "Rainbow Sphinx"\n',
 'review/helpfulness: 7/7\n',
 'review/score: 3.0\n',
 'review/time: 1182729600\n',
 'review/summary: "There Is So Much Darkness Now ~ Come For The Miracle"\n',
 'review/text: Synopsis: On the daily trek from Juarez, Mexico to El Paso, Texas an ever increasing number of female workers are found raped and murdered in the surrounding desert. Investigative reporter Karina Danes (Minnie Driver) arrives from Los Angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truth.<br /><br />Her story goes nationwide when a young girl named Mariela (Ana Claudia Talancon) survives a vicious attack and walks out of the desert crediting the Blessed Virgin for her rescue. Her story is further enhanced when the "Wounds of Christ" (stigmata) appear in her 

This quick view into the read text file shows us that it consists of a list of 71205231 lines, each of them being a string and corresponding to one attribute of a review. There are 8 different attributes, ordered in a cyclic manner. Each 9th element is a "\n" (new line) line. Each of the lines ends in a "\n".

### Getting the Reviews

In [6]:
# Keeping only the lines which contain the product ID, the reviewer ID and the rating (filtering out all other lines)
clean_reviews = [line for line in lines if re.findall(r"product/productId|review/userId|review/score", line) != []]

In [14]:
# Quick look into the filtered dataset
# Only the lines with one of the tree types of attributes mentioned above are kept
print(clean_reviews[:9])

['product/productId: B003AI2VGA\n', 'review/userId: A141HP4LYPWMSR\n', 'review/score: 3.0\n', 'product/productId: B003AI2VGA\n', 'review/userId: A328S9RN3U5M68\n', 'review/score: 3.0\n', 'product/productId: B003AI2VGA\n', 'review/userId: A1I7QGUDP043DG\n', 'review/score: 5.0\n']


In [8]:
# Counting the number of each of the three categories (attribute and category are used interchangeably)
num_prodcts = len([line for line in clean_reviews if "product/productId" in line])
num_users = len([line for line in clean_reviews if "review/userId" in line])
num_score = len([line for line in clean_reviews if "review/score" in line])

In [9]:
# Checking if there are the same amount of elements in each category (roughly checking that reviews are complete)
if num_prodcts == num_users == num_score:
    print(num_prodcts, "complete reviews with each three pieces of information")

7911684 complete reviews with each three pieces of information


In [10]:
# Cleaning the remaining lines, removing "\n" at end of the lines and category names at the beginning of each line
cat_list = ['product/productId: ', 'review/userId: ', 'review/score: ', '\n']   
final = [re.sub("|".join(cat_list), "", line) for line in clean_reviews]

In [13]:
# The cleaned review elements
print(final[:9])

['B003AI2VGA', 'A141HP4LYPWMSR', '3.0', 'B003AI2VGA', 'A328S9RN3U5M68', '3.0', 'B003AI2VGA', 'A1I7QGUDP043DG', '5.0']


In [16]:
# ! This code line takes long to run !
# Grouping lines into list of three subsequent elements corresponding to one review
chunks = [final[i:i+3] for i in range(0, len(final), 3)]

In [18]:
# Quick look at the new format
chunks[:3]

[['B003AI2VGA', 'A141HP4LYPWMSR', '3.0'],
 ['B003AI2VGA', 'A328S9RN3U5M68', '3.0'],
 ['B003AI2VGA', 'A1I7QGUDP043DG', '5.0']]

At this point we have filtered out unnecessary information and possess a list of lists, where each of the lists (chunks) has size 3 and corresponds to the three relevant elements to one review.

This following code is a quicker way to split the long list of lines into reviews, however it was elaborated after the dataset was already fully processed and was thus not used. It is just here for reference.

### Exploring the Data

In [19]:
# Counting the number of different movies, users and ratings
# Movie is always the first element in the review list, user the second and rating the third
all_movies = set([rvw[0] for rvw in chunks])
all_users = set([rvw[1] for rvw in chunks])
all_stars = set([rvw[2] for rvw in chunks])

In [20]:
print("Number of different movies reviewed:", len(all_movies))
print("Number of different users:", len(all_users))
print("Number of different scores:", len(all_stars))

Number of different movies reviewed: 253059
Number of different users: 889176
Number of different scores: 5


In [21]:
# Look at which ratings are possible: ratings go from 1 to 5
all_stars

{'1.0', '2.0', '3.0', '4.0', '5.0'}

Next, we need to filter out the bad reviews. We had two approaches; a soft one, considering reviews as "positive" when they had 3 or more as rating and a hard one, where only 4 and 5 rating reviews were kept. For the final dataset, we made use of the hard approach since there was still largely enough data choosing this method and also because "positiveness" was thought to be better represented than the average rating 3.

In [22]:
# Filtering out the bad reviews (= 1 or 2): soft method
good_reviews = [rvw for rvw in chunks if rvw[2]!='1.0' and rvw[2]!='2.0']
len(good_reviews)

6826953

In [23]:
# ALternative filtering where good reviews only 4 or 5 (hard method)
good_reviews_alt = [rvw for rvw in good_reviews if rvw[2]!='3.0']
len(good_reviews_alt)

6035359

### Reading the Dataset into a Dataframe
The next step is done for efficiency purposes, since working with a dataframe is computationally way faster.

In [24]:
# Create a dataframe with the data
df = pd.DataFrame(good_reviews_alt, columns=["Movie", "User", "Score"])

In fact after this step we could have discarded the column with the score since we only needed the ratings to filter out the bad reviews, but the storage in the dataframe is efficient and will not significantly hinder the computations in the next steps so we decided to keep this information in for clarity reasons.

In [25]:
# Looking at the dataframe
df.head(10)

,Movie,User,Score
0,B003AI2VGA,A1I7QGUDP043DG,5.0
1,B00006HAXW,AD4CDZK7D31XP,5.0
2,B00006HAXW,A3Q4S5DFVPB70D,5.0
3,B00006HAXW,A2P7UB02HAVEPB,5.0
4,B00006HAXW,A2TX99AZKDK0V7,4.0
5,B00006HAXW,AFC8IKR407HSK,5.0
6,B00006HAXW,A1FRPGQYQTAOR1,5.0
7,B00006HAXW,A1RSDE90N6RSZF,5.0
8,B00006HAXW,A1OUBOGB5970AO,4.0
9,B00006HAXW,A3NPHQVIY59Y0Y,5.0


In [26]:
# Number of times one user appears (= how many reviews by the user)
num_rvw_users = df['User'].value_counts()

In [27]:
# Number of different reviewers
print(len(num_rvw_users))

738205


In [30]:
# Reviewers with most reviews
num_rvw_users[:10]

A16CZRQL23NOIW    10151
A2NJO6YE954DBH     8127
A10ODC971MDHV8     7548
A35ZK3M8L9JUPX     7507
A39CX0EE4BZCZC     6641
ANCOMAI0I7LVG      6585
A328S9RN3U5M68     6085
A3LZGLA88K0LA0     5880
A3UDYY6L2NH3JS     5737
AIMR915K4YCN       5674
Name: User, dtype: int64

In [31]:
# Create a dictionary of user as key and number of reviews as value
dict_user = dict(num_rvw_users)

In [32]:
# Getting the keys of all reviewers with more than one review
over_one = [key for key in dict_user.keys() if dict_user[key]!=1]
len(over_one)

505797

### The Final Dataframe

In [33]:
# The final preprocessed and filtered dataframe with only the good reviews and users with more than one review
final_df = df[df['User'].isin(over_one)].reset_index(drop=True)

In [34]:
# Some general information on the dataset
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5802951 entries, 0 to 5802950
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Movie   object
 1   User    object
 2   Score   object
dtypes: object(3)
memory usage: 132.8+ MB


In [35]:
final_df.head()

,Movie,User,Score
0,B003AI2VGA,A1I7QGUDP043DG,5.0
1,B00006HAXW,AD4CDZK7D31XP,5.0
2,B00006HAXW,A3Q4S5DFVPB70D,5.0
3,B00006HAXW,A2P7UB02HAVEPB,5.0
4,B00006HAXW,A2TX99AZKDK0V7,4.0


In [36]:
# Number of reviews left in the processed dataset
len(final_df)

5802951

In [ ]:
# Saving the dataframe as a seperate .CSV file (to use later on)
final_df.to_csv('PositiveReviews.csv')

## Prepare the Dataset with the Categories
The aim of this section is to merge the above pre-processed dataset with another set retrieved from GitHub containing the categories under which the movies were place on Amazon.com.

In [37]:
# Read in the dataset from a .CSV file
genre_df = pd.read_csv("labels.csv", names=['Movie', 'Category'])
genre_df.head()

,Movie,Category
0,B0029Z8KCY,"['Movies & TV', 'Boxed Sets', 'Anime']"
1,B00004CQT3,"['Movies & TV', 'Genre for Featured Categories..."
2,B00004CQT4,"['Movies & TV', 'Genre for Featured Categories..."
3,B006JIUN2W,[]
4,B004MPGBHK,[]


In [38]:
# Look at the amount of different categories/website paths the movies were placed in
len(set(genre_df['Movie']))

253059

In [39]:
# Filter out items where no category is given
cleaned_genre = genre_df.loc[(genre_df['Category']!='[]')].reset_index(drop=True)
cleaned_genre.head()

,Movie,Category
0,B0029Z8KCY,"['Movies & TV', 'Boxed Sets', 'Anime']"
1,B00004CQT3,"['Movies & TV', 'Genre for Featured Categories..."
2,B00004CQT4,"['Movies & TV', 'Genre for Featured Categories..."
3,B000009DX2,"['Movies & TV', 'Art House & International']"
4,B0071AD95K,"['Movies & TV', 'Genre for Featured Categories..."


In [40]:
# All unique category labels after having cleaned as above
labels = set(list(cleaned_genre["Category"]))
# All the category labels after having cleaned (non-unique)
all_labels = list(cleaned_genre['Category'])

In [41]:
# Cleaning the category labels since they are single strings containing non-useful symbols such as "[" to transform them 
# into a list of strings
clean_lbl = []
for label in labels:
    lbl = re.sub("\[|\]|\"|\'", "", label) 
    clean_lbl.append(lbl.split(","))

In [42]:
# Examples of the cleaned labels
clean_lbl[:3]

[['Movies & TV',
  ' Studio Specials',
  ' Miramax Home Entertainment',
  ' Foreign Spotlight'],
 ['Arts',
  ' Crafts & Sewing',
  ' Scrapbooking & Stamping',
  ' Scrapbooking Tools'],
 ['Movies & TV', ' FX', ' All FX Shows']]

In [43]:
# Looking at the different largest categories (first element in list of labels, corresponding to hierarchical order)
all_cat = set([lbl[0] for lbl in clean_lbl])
# The different main categories under which the items are found
print(all_cat)

{'Books', 'Clothing', 'Video Games', 'Tools & Home Improvement', 'Toys & Games', 'Home & Kitchen', 'Grocery & Gourmet Food', 'Musical Instruments', 'Beauty & Personal Care', 'Electronics', 'Health & Household', 'Pet Supplies', 'Patio', 'Sports & Outdoors', 'Cell Phones & Accessories', 'Industrial & Scientific', 'Arts', 'CDs & Vinyl', 'Office Products', 'Movies & TV', 'Automotive', 'Software'}


In [44]:
# Keeping only the items of the "Movies & TV" category seems sensible
movies = [lbl for lbl in clean_lbl if lbl[0]=="Movies & TV"]
# The amount of different categories within the Movies and TV category
print(len(movies))

715


In [45]:
# Same cleaning procedure for all labels (not only unique ones)
clean = [re.sub("\[|\]|\"|\'", "", label) for label in all_labels]

In [46]:
# Adding the cleaned categories to the dataframe
cleaned_genre['CleanCategories'] = [label.split(',') for label in clean]
cleaned_genre.head()

,Movie,Category,CleanCategories
0,B0029Z8KCY,"['Movies & TV', 'Boxed Sets', 'Anime']","[Movies & TV, Boxed Sets, Anime]"
1,B00004CQT3,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ..."
2,B00004CQT4,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ..."
3,B000009DX2,"['Movies & TV', 'Art House & International']","[Movies & TV, Art House & International]"
4,B0071AD95K,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ..."


In [47]:
# Final dataframe with for each review a list of strings corresponding to different categories (in which often genre 
# information is included), filtering out all items that are not in the "Movies and TV" category
final_genre = cleaned_genre[cleaned_genre['CleanCategories'].isin(movies)].reset_index(drop=True)
final_genre

,Movie,Category,CleanCategories
0,B0029Z8KCY,"['Movies & TV', 'Boxed Sets', 'Anime']","[Movies & TV, Boxed Sets, Anime]"
1,B00004CQT3,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ..."
2,B00004CQT4,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ..."
3,B000009DX2,"['Movies & TV', 'Art House & International']","[Movies & TV, Art House & International]"
4,B0071AD95K,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ..."
...,...,...,...
207880,B003FZW7VC,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ..."
207881,B00447L4KA,"['Movies & TV', 'Studio Specials', 'Warner Hom...","[Movies & TV, Studio Specials, Warner Home V..."
207882,B003S1UNZU,"['Movies & TV', 'Boxed Sets', 'Documentary']","[Movies & TV, Boxed Sets, Documentary]"
207883,B00008G1Y9,"['Movies & TV', 'Studio Specials', 'Warner Hom...","[Movies & TV, Studio Specials, Warner Home V..."


## Towards Merging the Information of both Datasets

In [48]:
# The unique movies present in the Amazon Movie Reviews dataset (df1) and in the extended dataset with categories (df2)
lst_movies_df1 = set(final_df['Movie'])
lst_movies_df2 = set(final_genre['Movie'])

In [49]:
# Getting a list of the movies that are in the extended genre dataframe but not in the main pre-processed one
not_movies = list(lst_movies_df1 - lst_movies_df2)
# Amount of movies not in df1 but in df2
len(not_movies)

38399

In [50]:
# Keep only the rows of the pre-processed dataframe if the movie is also in the genre dataframe (for comparability)
clean_final = final_df[final_df['Movie'].isin(lst_movies_df2)].reset_index(drop=True)
clean_final

,Movie,User,Score
0,B00004CQT3,A34KFDQ5KBHZA5,5.0
1,B00004CQT3,A1CIW2OEVAJRM2,5.0
2,B00004CQT3,A1VJCDRXUQVXBM,5.0
3,B00004CQT3,A2IMLPUXYQJTSY,5.0
4,B00004CQT3,A1D12NAC1U12F0,5.0
...,...,...,...
4802661,B000GRUN4A,A3H4EBR4RZIB9Y,5.0
4802662,B003F32YGC,AOI2LGQR941L7,5.0
4802663,B003F32YGC,ACDKM2C99SFCP,5.0
4802664,6304952198,A23KKLV2CD39U8,4.0


In [51]:
# Next we finally merge both dataframes on the movie names (product ID)
complete_df = pd.merge(final_genre, clean_final)
complete_df.columns

Index(['Movie', 'Category', 'CleanCategories', 'User', 'Score'], dtype='object')

In [52]:
# Look at the final dataframe
complete_df.head(5)

,Movie,Category,CleanCategories,User,Score
0,B0029Z8KCY,"['Movies & TV', 'Boxed Sets', 'Anime']","[Movies & TV, Boxed Sets, Anime]",AFV2584U13XP3,4.0
1,B00004CQT3,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ...",A34KFDQ5KBHZA5,5.0
2,B00004CQT3,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ...",A1CIW2OEVAJRM2,5.0
3,B00004CQT3,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ...",A1VJCDRXUQVXBM,5.0
4,B00004CQT3,"['Movies & TV', 'Genre for Featured Categories...","[Movies & TV, Genre for Featured Categories, ...",A2IMLPUXYQJTSY,5.0


In [53]:
# Additional information on number of reviews left (and attributes per review)
complete_df.shape

(4802666, 5)

### Saving the Data for Easier Accessibility

In [ ]:
complete_df.to_csv('CompleteDataWithGenres.csv')

In [54]:
# A smaller sample for network elaboration
small_df = complete_df.sample(frac=0.01, random_state=1)
len(small_df)

48027

In [ ]:
small_df.to_csv('1%SampleDataset.csv')

## Reducing the Dataset Size by Genre
It should be noted that even after the different pre-processing steps the dataset is still too large for the network computation in a reasonable time. Therefore, in the following part we try to reduce the dataset size to 10% of the whole dataset for those practical reasons. In order to do so we need to clean further the categories of each movie and select some few genres for the 10% datasample.

In [ ]:
# Quick way to read in data
all_data = pd.read_csv("CompleteDataWithGenres.csv") 

In [63]:
all_data.head()

,Movie,CleanCategories,User,Score
0,B0029Z8KCY,"['Movies & TV', ' Boxed Sets', ' Anime']",AFV2584U13XP3,4.0
1,B00004CQT3,"['Movies & TV', ' Genre for Featured Categorie...",A34KFDQ5KBHZA5,5.0
2,B00004CQT3,"['Movies & TV', ' Genre for Featured Categorie...",A1CIW2OEVAJRM2,5.0
3,B00004CQT3,"['Movies & TV', ' Genre for Featured Categorie...",A1VJCDRXUQVXBM,5.0
4,B00004CQT3,"['Movies & TV', ' Genre for Featured Categorie...",A2IMLPUXYQJTSY,5.0
...,...,...,...,...
4802661,B00008G1Y9,"['Movies & TV', ' Studio Specials', ' Warner H...",A3IM77LLZAN8B6,5.0
4802662,B00008G1Y9,"['Movies & TV', ' Studio Specials', ' Warner H...",A64I34EBP1OMO,4.0
4802663,B00008G1Y9,"['Movies & TV', ' Studio Specials', ' Warner H...",A2YSDC0GAA7MV0,4.0
4802664,B003F32YGC,"['Movies & TV', ' Genre for Featured Categorie...",AOI2LGQR941L7,5.0


In [56]:
# All different movie categories within the "Movies & TV" category
type_movies = [mv[1:] for mv in movies]

In [89]:
# Transform strings of categories into lists of tokens
catgrs = []
for cat in tqdm(list(all_data["CleanCategories"])):
    lbl = re.sub("\[|\]|\"|\'", "", cat)
    catgrs.append(lbl.split(","))

100%|██████████████████████████████████████████████████████████████████████| 4802666/4802666 [09:25<00:00, 8490.65it/s]


In [107]:
# Look into the format
catgrs[:5]

[['  Boxed Sets', '  Anime'],
 ['  Genre for Featured Categories', '  Kids & Family'],
 ['  Genre for Featured Categories', '  Kids & Family'],
 ['  Genre for Featured Categories', '  Kids & Family'],
 ['  Genre for Featured Categories', '  Kids & Family']]

In [109]:
# All the category labels as one list
final_all_clean = []
# All the category labels as a list of lists for each review
final_for_df = []
for mv in tqdm(catgrs):
    lbls = []
    for lbl in mv:
        l = lbl.strip()
        lbls.append(l)
        final_all_clean.append(l)
    final_for_df.append(lbls)

100%|██████████████████████████████████████████████████████████████████████| 4802666/4802666 [09:31<00:00, 8406.32it/s]


In [110]:
# Look into the final format
final_for_df[:5]

[['Boxed Sets', 'Anime'],
 ['Genre for Featured Categories', 'Kids & Family'],
 ['Genre for Featured Categories', 'Kids & Family'],
 ['Genre for Featured Categories', 'Kids & Family'],
 ['Genre for Featured Categories', 'Kids & Family']]

After having gone manually through the approximately ~700 different labels (see All _genre_ categories excel format.xlsx) we observed that many of the labels do not make sense as genre. Therefore, we decided to look at the most frequent labels and choose from them some genre categories.

In [111]:
# Count the number of occurences of each label and look at the most frequent ones
counts = Counter(final_all_clean)
counts.most_common(25)

[('Studio Specials', 1805893),
 ('Genre for Featured Categories', 1317997),
 ('All Titles', 566628),
 ('Warner Home Video', 518428),
 ('Drama', 469854),
 ('Action & Adventure', 440784),
 ('Science Fiction & Fantasy', 393545),
 ('Comedy', 391450),
 ('Science Fiction', 378263),
 ('20th Century Fox Home Entertainment', 363494),
 ('Sony Pictures Home Entertainment', 312043),
 ('All Sony Pictures Titles', 303072),
 ('Universal Studios Home Entertainment', 286539),
 ('All Universal Studios Titles', 286539),
 ('General', 224106),
 ('Paramount Home Entertainment', 201875),
 ('Animation', 198698),
 ('Art House & International', 184574),
 ('Action', 148033),
 ('MGM Home Entertainment', 139420),
 ('All MGM Titles', 137911),
 ('Kids & Family', 131610),
 ('Musicals', 122976),
 ('All Fox Titles', 119921),
 ('Lionsgate Home Entertainment', 112709)]

After having compiled a list with 10 very frequent labels that were sensible as genre:
['Drama','Action & Adventure', 'Science Fiction & Fantasy', 'Comedy', 'Animation','Horror', 'Documentary', 'Musicals', 'Sports', 'Kids & Family']
<p>About 44.56% of the data was still included (2140243 reviews), which for the purpose of our network creation is computationally too big.
We therefore selected five genres, that were a little less frequent and for further analysis were intuitively well distinct from each other. This left us with about 10% of the data, which is the one we will use for the rest of the project.

In [112]:
# The chosen genres
defined_genres = ['Action', 'Horror', 'Documentary', 'Musicals', 'Anime & Manga']

In [113]:
# Reducing each label list for each review to one of the elements in the defined genres
# Giving "NaN" value if none of the labels correspond to one of the genre and choosing randomly one genre if a movie has 
# more than one of the defined genres within its label list
genres = []
for lbl in tqdm(final_for_df):
    if any(gnre in lbl for gnre in defined_genres): # checking if the movie is one of the genres
        g = set(lbl) & set(defined_genres) # get the overlapping element(s) of the label list and the genres
        if len(g)==1: # if only one genre overlap
            genres.append("".join(g))
        else: # if more than one overlap then choose randomly
            genres.append("".join(random.sample(g, 1)))
    else: # none if no overlap
        genres.append("NaN")

100%|████████████████████████████████████████████████████████████████████| 4802666/4802666 [00:08<00:00, 590865.68it/s]


In [117]:
# Look at the format
print(genres[25:70])

['NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'Documentary', 'Documentary', 'Documentary', 'Documentary', 'Documentary', 'Documentary', 'Documentary', 'Documentary', 'Documentary', 'Documentary', 'Documentary', 'Horror', 'Horror', 'Horror', 'Horror', 'Horror', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN']


In [118]:
# Replace the categories by the genres list and rename the column as genre
all_data['CleanCategories'] = genres
all_data.rename(columns = {'CleanCategories':'Genre'}, inplace=True)

In [120]:
# Drop out all rows if the genre is "NaN" (none of the chosen genre)
all_data = all_data[all_data.Genre != "NaN"].reset_index(drop=True)
all_data.drop(["Unnamed: 0"], axis=1) # drop this additional column

,Movie,Genre,User,Score
0,B000NDFLWG,Documentary,A3GYVMUS7SSDFW,5.0
1,B000NDFLWG,Documentary,AJYGQV81FSFE2,4.0
2,B000NDFLWG,Documentary,A39F5HLWQEZD7E,5.0
3,B000NDFLWG,Documentary,A21SBJ6QU1C4IX,5.0
4,B000NDFLWG,Documentary,A1TK6WNUIAEQRU,4.0
...,...,...,...,...
468739,B003S1UNZU,Documentary,A3KP81PWE0JH5E,5.0
468740,B003S1UNZU,Documentary,A3AD071UEP20WS,5.0
468741,B003S1UNZU,Documentary,A3IPZPME7QYQS6,5.0
468742,B003S1UNZU,Documentary,A1PNWMXODZ049K,5.0


In [ ]:
# Save the final dataset
all_data.to_csv('10%Dataset.csv')

## Dictionary with Movie as Key and Genre as Value
This dictionary will be needed for the percentage analysis of the clusters, again using the 10% sample.

In [121]:
# The data that will be used
data_for_clusters =  pd.read_csv("10%Dataset.csv") 

In [123]:
# Keep only the two categories of interest
data_for_clusters = data_for_clusters[["Movie", "MovieGenre"]]
data_for_clusters.head(10)

,Movie,MovieGenre
0,B000NDFLWG,Documentary
1,B000NDFLWG,Documentary
2,B000NDFLWG,Documentary
3,B000NDFLWG,Documentary
4,B000NDFLWG,Documentary
5,B000NDFLWG,Documentary
6,B000NDFLWG,Documentary
7,B000NDFLWG,Documentary
8,B000NDFLWG,Documentary
9,B000NDFLWG,Documentary


In [124]:
# Dropping duplicate rows which in this case correspond to movie names which have supposedly always the same genre
dataclusters = data_for_clusters.drop_duplicates().reset_index(drop=True)
dataclusters.head(10)

,Movie,MovieGenre
0,B000NDFLWG,Documentary
1,B008FPU7AA,Horror
2,1562229567,Documentary
3,1888617047,Musicals
4,6305508569,Action
5,6304474415,Documentary
6,B0006FFRD4,Anime & Manga
7,B003BUAP10,Documentary
8,B000BB1NFO,Documentary
9,B00020HBN2,Anime & Manga


Next, we check that there are indeed only unique movie names left since in the function before we used a "random" function
for choosing a genre when a movies possesses more than 1 of the selected genres. In fact, this random choice could make it happen that a movie is kept in the dataframe since it was assigned two different genres and is thus not filtered out as a duplicate.

In [125]:
# The number of different movies left at last
len(dataclusters)

21788

In [127]:
# Check that no movie is kept twice with different genre
# Since the length of the unique movies in the dataframe is the same as all movies in the dataframe, all movies are unique
len(dataclusters['Movie'].value_counts())

21788

In [128]:
# Preparing the dictionary
keys = dataclusters["Movie"]
values = dataclusters["MovieGenre"]

In [130]:
# The movie-genre dictionary
dict_mvs_genre = dict(zip(keys, values))
dict_mvs_genre

{'B000NDFLWG': 'Documentary',
 'B008FPU7AA': 'Horror',
 '1562229567': 'Documentary',
 '1888617047': 'Musicals',
 '6305508569': 'Action',
 '6304474415': 'Documentary',
 'B0006FFRD4': 'Anime & Manga',
 'B003BUAP10': 'Documentary',
 'B000BB1NFO': 'Documentary',
 'B00020HBN2': 'Anime & Manga',
 'B000GAKJXM': 'Documentary',
 'B00020HBNW': 'Anime & Manga',
 'B00020HBNC': 'Anime & Manga',
 'B0012EM5GK': 'Anime & Manga',
 'B0002V7TJM': 'Documentary',
 'B0002V7TJC': 'Documentary',
 'B00004R7CH': 'Musicals',
 'B0002V7TJW': 'Documentary',
 'B00000IC80': 'Musicals',
 '6300246299': 'Horror',
 'B00000IC8F': 'Musicals',
 'B0014567LQ': 'Horror',
 'B001DWNUBU': 'Documentary',
 'B008R52L4I': 'Horror',
 'B000H5U5IU': 'Horror',
 'B000VZC4XA': 'Horror',
 'B0071BY2YQ': 'Documentary',
 '0764004492': 'Horror',
 'B006HNRSFE': 'Documentary',
 'B0007M21Y4': 'Musicals',
 '6300266850': 'Horror',
 'B000KGGJ04': 'Action',
 'B000A9QK8M': 'Action',
 'B003ZZ7TT2': 'Musicals',
 'B000FP2PE4': 'Documentary',
 '6304462689'